In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



Using TensorFlow backend.


In [2]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
    'model_name': TelegramData.name
}

tg_callback = TelegramCallback(config)

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [4]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14,15,16]
test_ids = [4,5,6,11,17]

In [24]:
dfAll = pd.read_pickle("PklData/df_lstm2.pkl")

In [25]:
dfAll.head()

,userID,TaskID,VersionID,Blobs,BlobCount
0,15,9,2,"[[[0, 1, 0, 2, 0, 160, 1, 1, 3, 1, 1, 0, 0, 0,...",7
1,15,9,3,"[[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0...",15
2,15,9,4,"[[[1, 1, 2, 2, 2, 1, 0, 0, 0, 1, 173, 1, 1, 0,...",11
3,15,9,5,"[[[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0...",12
4,15,9,6,"[[[0, 0, 0, 0, 0, 0, 0, 146, 163, 1, 0, 0, 1, ...",9


In [28]:
df_train.shape

(6995, 6)

In [54]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_test = df_test.reset_index()
df_train = df_train.reset_index()

df_train2 = df_train[['Blobs', 'TaskID']].copy()
df_test2 = df_test[['Blobs', 'TaskID']].copy()

In [71]:
df_train.shape

(6995, 6)

In [56]:
dfAll.head()

,userID,TaskID,VersionID,Blobs,BlobCount
0,15,9,2,"[[[0, 1, 0, 2, 0, 160, 1, 1, 3, 1, 1, 0, 0, 0,...",7
1,15,9,3,"[[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0...",15
2,15,9,4,"[[[1, 1, 2, 2, 2, 1, 0, 0, 0, 1, 173, 1, 1, 0,...",11
3,15,9,5,"[[[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0...",12
4,15,9,6,"[[[0, 0, 0, 0, 0, 0, 0, 146, 163, 1, 0, 0, 1, ...",9


In [70]:
x_train.shape

(111426, 1, 27, 15, 1)

In [68]:
x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,50,27,15,1)
x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,50,27,15,1)

y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values



ValueError: cannot reshape array of size 45127530 into shape (50,27,15,1)

In [69]:
y_train.shape

(6995,)

In [31]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(dfAll.TaskID.unique())
y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)

In [64]:
batch_size = 50
epochs = 100
timesteps = 1
data_dim = (27,15)
with tf.device('/device:GPU:1'):
    config = tf.ConfigProto(log_device_placement = True, allow_soft_placement = True)
    config.gpu_options.allow_growth=True
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    config.gpu_options.allocator_type = 'BFC'
    
    with tf.Session(config=config):
        tf.get_default_graph()
        model = Sequential()
        model.add(TimeDistributed(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu'), input_shape=(None, 27, 15, 1)))
        model.add(TimeDistributed(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu')))
        model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
        model.add(TimeDistributed(Dropout(0.5)))

        model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu')))
        model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu')))
        model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
        model.add(TimeDistributed(Dropout(0.5)))

        model.add(TimeDistributed(Flatten()))

        model.add(LSTM(75, return_sequences=True, input_shape=(timesteps, data_dim)))
        model.add(LSTM(75))
        model.add(Dense(num_classes))

        optimizer = optimizers.Adam(lr = 0.001, decay=1e-6)
        #optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
        model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])

        # Broadcast progress to the tensorboard.
        readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
        tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp

        config = ""
        for layer in model.layers:
            config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
        config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n"

        model.summary()

        readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
        tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp
        print("KnuckleFinger_LSTM_Jan_" + readable_timestamp)
        logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0,
                                    write_graph=True, write_images=True, update_freq = 'epoch')
        storer = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_LSTM_Jan_" + readable_timestamp + ".h5", monitor='val_loss', verbose=0,
                                 save_best_only=True, save_weights_only=False, mode='auto', period=1)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                                    patience=10, 
                                                    verbose=1, 
                                                    factor=0.95, 
                                                    min_lr=0.00001)

        history = model.fit(x_train, y_train_one_hot,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_test, y_test_one_hot),
                                callbacks=[storer,tg_callback, learning_rate_reduction])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_37 (TimeDis (None, None, 27, 15, 32)  320       
_________________________________________________________________
time_distributed_38 (TimeDis (None, None, 27, 15, 32)  9248      
_________________________________________________________________
time_distributed_39 (TimeDis (None, None, 13, 7, 32)   0         
_________________________________________________________________
time_distributed_40 (TimeDis (None, None, 13, 7, 32)   0         
_________________________________________________________________
time_distributed_41 (TimeDis (None, None, 13, 7, 64)   18496     
_________________________________________________________________
time_distributed_42 (TimeDis (None, None, 13, 7, 64)   36928     
_________________________________________________________________
time_distributed_43 (TimeDis (None, None, 6, 3, 64)    0         
__________

ValueError: Input arrays should have the same number of samples as target arrays. Found 111426 input samples and 6995 target samples.